In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data_dir = "/content/drive/MyDrive/Tomato_leafs/plant disease"

In [2]:
train_dir='/content/drive/MyDrive/Tomato_leafs/plant disease/train'

In [3]:
test_dir='/content/drive/MyDrive/Tomato_leafs/plant disease/valid'

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test=ImageDataGenerator(rescale=1./255)

In [5]:
traindata=train.flow_from_directory(train_dir,target_size=(128,128),batch_size=32,class_mode='categorical')

Found 1555 images belonging to 5 classes.


In [6]:
testdata=test.flow_from_directory(test_dir,target_size=(128,128),batch_size=32,class_mode='categorical')

Found 473 images belonging to 5 classes.


In [7]:
import tensorflow as tf

def depthwise_separable_conv(input, filters, kernel_size, strides):
    x = tf.keras.layers.DepthwiseConv2D(kernel_size=kernel_size, strides=strides, padding='same')(input)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=1, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x

def mobilenet(input_shape, num_classes):
    input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, kernel_size=3, strides=(2,2), padding='same')(input)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = depthwise_separable_conv(x, filters=64, kernel_size=3, strides=(1,1))
    x = depthwise_separable_conv(x, filters=128, kernel_size=3, strides=(2,2))
    x = depthwise_separable_conv(x, filters=128, kernel_size=3, strides=(1,1))
    x = depthwise_separable_conv(x, filters=256, kernel_size=3, strides=(2,2))
    x = depthwise_separable_conv(x, filters=256, kernel_size=3, strides=(1,1))
    x = depthwise_separable_conv(x, filters=512, kernel_size=3, strides=(2,2))

    for i in range(5):
        x = depthwise_separable_conv(x, filters=512, kernel_size=3, strides=(1,1))

    x = depthwise_separable_conv(x, filters=1024, kernel_size=3, strides=(2,2))
    x = depthwise_separable_conv(x, filters=1024, kernel_size=3, strides=(1,1))

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=input, outputs=output)
    return model


In [8]:
model = mobilenet(input_shape=(224, 224, 3), num_classes=5)


In [9]:
import keras
from keras.optimizers import Adam
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [10]:
var=model.fit(traindata,validation_data=testdata,epochs=5,batch_size=32)

Epoch 1/5
49/49 [==============================] - 360s 7s/step - loss: 1.4785 - accuracy: 0.3222 - val_loss: 1.6415 - val_accuracy: 0.2093
Epoch 2/5
49/49 [==============================] - 12s 236ms/step - loss: 1.3721 - accuracy: 0.3820 - val_loss: 1.7253 - val_accuracy: 0.1332
Epoch 3/5
49/49 [==============================] - 12s 249ms/step - loss: 1.2515 - accuracy: 0.4695 - val_loss: 1.8644 - val_accuracy: 0.1332
Epoch 4/5
49/49 [==============================] - 12s 250ms/step - loss: 1.1371 - accuracy: 0.5023 - val_loss: 2.0400 - val_accuracy: 0.1332
Epoch 5/5
49/49 [==============================] - 12s 247ms/step - loss: 0.9798 - accuracy: 0.5910 - val_loss: 2.2661 - val_accuracy: 0.1332
